In [1]:
#web scraper

import glob
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import numpy as np

url = 'https://www.alexa.com/topsites/countries/KR'

html_website_ranking = requests.get(url).text
#requests.get(url)을 통해 url의 html을 '요청'함 - 응답(Response)이 돌아옴
#이를 확인하려면 .text 달아주면 html 전체 보여줌
soup_website_ranking = BS(html_website_ranking, 'lxml')

website_ranking = soup_website_ranking.select('p a')
del website_ranking[0]

In [2]:
website_ranking[:6]

[<a href="/siteinfo/google.com">Google.com</a>,
 <a href="/siteinfo/naver.com">Naver.com</a>,
 <a href="/siteinfo/youtube.com">Youtube.com</a>,
 <a href="/siteinfo/daum.net">Daum.net</a>,
 <a href="/siteinfo/tistory.com">Tistory.com</a>,
 <a href="/siteinfo/tmall.com">Tmall.com</a>]

In [3]:
website_ranking[0].get_text()

'Google.com'

In [4]:
website_ranking_address = [element.get_text() for element in website_ranking]

In [5]:
website_ranking_address[:6]

['Google.com',
 'Naver.com',
 'Youtube.com',
 'Daum.net',
 'Tistory.com',
 'Tmall.com']

In [6]:
website_ranking_dict = {'Website': website_ranking_address}
df = pd.DataFrame(website_ranking_dict, columns = ['Website'],
                 index = range(1, len(website_ranking_address) + 1))
df[:6]

,Website
1,Google.com
2,Naver.com
3,Youtube.com
4,Daum.net
5,Tistory.com
6,Tmall.com


In [7]:
#음악 순위 music ranking

url = 'https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=2017&month=12&week=1&page='
html_music = requests.get(url).text
soup_music = BS(html_music, 'lxml')

"""
<a href="#40987767" class="_title title NPI=a:track,r:1,i:40987767" title="다시 여기 바닷가"><span class="ellipsis">다시 여기 바닷가</span></a>
    <span class="ellipsis">다시 여기 바닷가</span>

=> a 태그 요소 중 class 속성값이 _title 인 것 찾고
    그 다음 span 태그의 ellipsis 요소를 추출
"""
titles = soup_music.select('a._title span.ellipsis')
titles[:7]

[<span class="ellipsis">눈 (Feat. 이문세)</span>,
 <span class="ellipsis">기억의 빈자리</span>,
 <span class="ellipsis">선물</span>,
 <span class="ellipsis">Beautiful</span>,
 <span class="ellipsis">좋아</span>,
 <span class="ellipsis">피카부 (Peek-A-Boo)</span>,
 <span class="ellipsis">좋니</span>]

In [8]:
music_titles = [title.get_text() for title in titles]

In [9]:
music_titles[:7]

['눈 (Feat. 이문세)', '기억의 빈자리', '선물', 'Beautiful', '좋아', '피카부 (Peek-A-Boo)', '좋니']

In [10]:
#artists 추출

artists = soup_music.select('a._artist span.ellipsis')
artists[0].get_text()

'\r\n\t\t\t\r\n\t\t\t\r\n\t\t\tZion.T\r\n\t\t'

In [11]:
music_artists = [artist.get_text().strip() for artist in artists]

In [12]:
music_artists[:7]
#5위 좋아의 artist 민서가 출력 안 됨

['Zion.T',
 '나얼',
 '멜로망스(Melomance)',
 'Wanna One(워너원)',
 'Red Velvet (레드벨벳)',
 '윤종신',
 '뉴이스트 W']

In [13]:
'''
artist 민서 요소 검사 해보면 다른 artist의 html 코드와 다름
다른 artist는
td class="_artist"
    a class="_artist"
        span class="ellipsis"
형식인 데 반해

민서 artist는
td class="_artist artist no_ell2"
    a class="NPI=a:layerbtn,r:5"
형식이므로

공통으로 존재하는 td 클래스와 a 클래스를 이용
'''
artists = soup_music.select('td._artist a')
artists[:7]

[<a class="_artist NPI=a:artist,r:1,i:115967" href="/artist/home.nhn?artistId=115967" title="Zion.T">
 <span class="ellipsis">
 			
 			
 			Zion.T
 		</span>
 </a>,
 <a class="_artist NPI=a:artist,r:2,i:9483" href="/artist/home.nhn?artistId=9483" title="나얼">
 <span class="ellipsis">
 			
 			
 			나얼
 		</span>
 </a>,
 <a class="_artist NPI=a:artist,r:3,i:377357" href="/artist/home.nhn?artistId=377357" title="멜로망스(Melomance)">
 <span class="ellipsis">
 			
 			
 			멜로망스(Melomance)
 		</span>
 </a>,
 <a class="_artist NPI=a:artist,r:4,i:1748968" href="/artist/home.nhn?artistId=1748968" title="Wanna One(워너원)">
 <span class="ellipsis">
 			
 			
 			Wanna One(워너원)
 		</span>
 </a>,
 <a alt="" class="NPI=a:layerbtn,r:5" href="javascript:void(0);" title="">민서</a>,
 <a class="_artist NPI=a:artist,r:6,i:343255" href="/artist/home.nhn?artistId=343255" title="Red Velvet (레드벨벳)">
 <span class="ellipsis">
 			
 			
 			Red Velvet (레드벨벳)
 		</span>
 </a>,
 <a class="_artist NPI=a:artist,r:7,i:182"

In [14]:
music_artists = [artist.get_text().strip() for artist in artists]

In [15]:
music_artists[:7]
#민서도 정상적으로 출력

['Zion.T',
 '나얼',
 '멜로망스(Melomance)',
 'Wanna One(워너원)',
 '민서',
 'Red Velvet (레드벨벳)',
 '윤종신']

In [16]:
for k in range(7):
    print("{0}: {1} / {2}".format(k+1, music_titles[k], music_artists[k]))

1: 눈 (Feat. 이문세) / Zion.T
2: 기억의 빈자리 / 나얼
3: 선물 / 멜로망스(Melomance)
4: Beautiful / Wanna One(워너원)
5: 좋아 / 민서
6: 피카부 (Peek-A-Boo) / Red Velvet (레드벨벳)
7: 좋니 / 윤종신


In [17]:
music_titles_artists = {}
order = 0

for (music_title, music_artist) in zip(music_titles, music_artists):
    order += 1
    music_titles_artists[order] = [music_title, music_artist]
    
#music_titles_artists = dict(zip(music_titles, music_artists))
#위 for 문은 순위를 key로 함 / 밑에는 간단하지만 title이 key가 됨

In [18]:
music_titles_artists

{1: ['눈 (Feat. 이문세)', 'Zion.T'],
 2: ['기억의 빈자리', '나얼'],
 3: ['선물', '멜로망스(Melomance)'],
 4: ['Beautiful', 'Wanna One(워너원)'],
 5: ['좋아', '민서'],
 6: ['피카부 (Peek-A-Boo)', 'Red Velvet (레드벨벳)'],
 7: ['좋니', '윤종신'],
 8: ['WHERE YOU AT', '뉴이스트 W'],
 9: ['LIKEY', 'TWICE(트와이스)'],
 10: ['그때의 나, 그때의 우리', '어반자카파'],
 11: ['있다면', '뉴이스트 W'],
 12: ['뻔한 이별 (PROD. 13)', '소유 (SOYOU..'],
 13: ['하루만', '뉴이스트 W'],
 14: ['여보세요', '뉴이스트'],
 15: ['DNA', '방탄소년단'],
 16: ['갖고 싶어', 'Wanna One(워너원)'],
 17: ['에너제틱 (Energetic)', 'Wanna One(워너원)'],
 18: ['특별해', '젝스키스'],
 19: ['연애소설 (Feat. 아이유)', '에픽하이 (EPIK HIGH)'],
 20: ['썸 탈꺼야', '볼빨간사춘기'],
 21: ['웃어줘', '젝스키스'],
 22: ['지금까지 행복했어요 (BAEKHO SOLO)', '뉴이스트 W'],
 23: ['그리워하다', '비투비'],
 24: ['GOOD LOVE (ARON SOLO)', '뉴이스트 W'],
 25: ['Twilight', 'Wanna One(워너원)'],
 26: ['밤편지', '아이유(IU)'],
 27: ['PARADISE (REN SOLO)', '뉴이스트 W'],
 28: ['WITH (JR SOLO)', '뉴이스트 W'],
 29: ['봄날', '방탄소년단'],
 30: ['Nothing Without You (Intro.)', 'Wanna One(워너원)'],
 31: ['아프지 마요', '젝스키스'],
 32: ['시차 (We 

In [19]:
music_titles_artists[1]

['눈 (Feat. 이문세)', 'Zion.T']

In [20]:
#반복되는 것을 함수로 정의
naver_music_url = 'https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=2017&month=12&week=1&page='
#TOP 100 차트 히스토리의 2017년 12월 1주차 차트, 마지막 &page= 을 붙여 열 page 선택 (+숫자만 해주면 됨, 두 페이지이므로 1 혹은 2)

def naver_music(url):
    html_music = requests.get(url).text
    soup_music = BS(html_music, 'lxml')
    
    titles = soup_music.select('a._title span.ellipsis')
    artists = soup_music.select('td._artist a')
    
    music_titles = [title.get_text() for title in titles]
    music_artists = [artist.get_text().strip() for artist in artists]
    
    return music_titles, music_artists

file_name = 'C:\\users\\Yubin\\mypy\\data\\NaverMusicTop100.txt'

f = open(file_name, 'w')

for page in range(2):
    naver_music_url_page = naver_music_url + str(page + 1)
    #naver_music_url + 1 혹은 2를 해주어 페이지 열기
    naver_music_titles, naver_music_artists = naver_music(naver_music_url_page)
    #url을 합친 뒤 함수에 삽입, return 되는 music_titles와 music_artists를 각각 리스트로 줌
    
    for k in range(len(naver_music_titles)):
        f.write('{0:2d}: {1}/{2}\n'.format(page*50 + k+1, naver_music_titles[k], naver_music_artists[k]))
        #한 페이지에 순위 50개 -> 계산해준 값이 {0}
f.close()

glob.glob(file_name)
#생성된 파일 확인

['C:\\users\\Yubin\\mypy\\data\\NaverMusicTop100.txt']

In [21]:
#확인 결과 82위의 All I Want For Christmas Is You 가 다른 항목들과 달라 출력 안 됨

#반복되는 것을 함수로 정의
naver_music_url = 'https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=2017&month=12&week=1&page='
#TOP 100 차트 히스토리의 2017년 12월 1주차 차트, 마지막 &page= 을 붙여 열 page 선택 (+숫자만 해주면 됨, 두 페이지이므로 1 혹은 2)

def naver_music(url):
    html_music = requests.get(url).text
    soup_music = BS(html_music, 'lxml')
    
    titles = soup_music.select('td.name span.ellipsis')
    #요소 검사 통해 확인한 결과 이렇게 하면 82위도 포함됨
    artists = soup_music.select('td._artist a')
    
    music_titles = [title.get_text() for title in titles]
    music_artists = [artist.get_text().strip() for artist in artists]
    
    return music_titles, music_artists

file_name = 'C:\\users\\Yubin\\mypy\\data\\NaverMusicTop100.txt'

f = open(file_name, 'w')

for page in range(2):
    naver_music_url_page = naver_music_url + str(page + 1)
    #naver_music_url + 1 혹은 2를 해주어 페이지 열기
    naver_music_titles, naver_music_artists = naver_music(naver_music_url_page)
    #url을 합친 뒤 함수에 삽입, return 되는 music_titles와 music_artists를 각각 리스트로 줌
    
    for k in range(len(naver_music_titles)):
        f.write('{0:2d}: {1}/{2}\n'.format(page*50 + k+1, naver_music_titles[k], naver_music_artists[k]))
        #한 페이지에 순위 50개 -> 계산해준 값이 {0}
f.close()

glob.glob(file_name)
#생성된 파일 확인

#체크해보니 100개 다 제대로 나옴

['C:\\users\\Yubin\\mypy\\data\\NaverMusicTop100.txt']

In [22]:
# p.439 이미지 가져오기
import requests

url = 'https://www.python.org/static/img/python-logo.png'
html_image = requests.get(url)
html_image
#Response [200]이 반환 = 이미지 파일 주소에 문제 없다는 의미
#앞에서는 .text 로 문자열 가져옴 - 이미지는?

<Response [200]>

In [23]:
import os

image_file_name = os.path.basename(url)
#basename 이용해 url에서 이미지 파일명만 추출
image_file_name

'python-logo.png'

In [24]:
folder = 'C:/users/Yubin/mypy/download'

#if not os.path.exists(folder):
#    os.makedirs(folder)

#os.path.exists로 folder가 이미 존재하는지 확인, 없으면 새로 만듦(makedirs)

In [25]:
image_path = os.path.join(folder, image_file_name)
#폴더 + 파일 이름
image_path

'C:/users/Yubin/mypy/download\\python-logo.png'

In [26]:
imageFile = open(image_path, 'wb')
#파일을 쓰기 + 바이너리 파일 모드로 지정

In [27]:
chunk_size = 1000000 #이미지 데이터를 1000000바이트씩 나눠서 내려받음
for chunk in html_image.iter_content(chunk_size):
    imageFile.write(chunk)
imageFile.close()

In [28]:
os.listdir(folder)
#지정 폴더의 파일 목록 보여주는 명령어

['giraffe_1.jpg', 'python-logo.png']

In [94]:
#여러 이미지 내려받기
#pixabay.com - 둘러보기 - Popular Images - 카테고리: 동물
#<img srcset="https://cdn.pixabay.com/photo/2020/07/21/22/30/heron-5427512__340.jpg 1x, https://cdn.pixabay.com/photo/2020/07/21/22/30/heron-5427512__480.jpg 2x" src="https://cdn.pixabay.com/photo/2020/07/21/22/30/heron-5427512__340.jpg" alt="헤론, 섭 새, 동물, 깃털, 부리, 서있는, 먹이, 물, 연못">
#위는 한 사진의 html - img 태그를 select 하면 사진 파일 가져올 수 있을 듯하다.

import requests
from bs4 import BeautifulSoup as BS

URL = 'https://picjumbo.com/'
#URL = 'https://www.google.com/search?q=giraffe&tbm=isch&ved=2ahUKEwitubig8O3qAhWPAaYKHam5DX0Q2-cCegQIABAA&oq=giraffe&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6BAgAEAM6CAgAELEDEIMBUJ8gWPpCYNlEaABwAHgBgAF3iAH_CZIBBDEuMTGYAQCgAQGqAQtnd3Mtd2l6LWltZ7ABAMABAQ&sclient=img&ei=FQQfX-3aEo-DmAWp87boBw&bih=615&biw=957&client=firefox-b-d'

html_google_image = requests.get(URL).text
soup_google_image = BS(html_google_image, 'lxml')
google_image_elements = soup_google_image.select('img')
google_image_elements[:3]

[<img alt="Free Stock Photos" height="34" src="https://picjumbo.com/wp-content/themes/picjumbofree/data/picjumbo_logo.png" width="130"/>,
 <img alt="Stock Photo Selection from iStock" src="https://picjumbo.com/wp-content/themes/picjumbofree/data/istock-logo.jpg?v=2.1"/>,
 <img alt="Download The Golden Gate Bridge Sunset FREE Stock Photo" class="image" data-src="https://picjumbo.com/wp-content/uploads/free-stock-photos-san-francisco-1080x720.jpg" itemprop="contentUrl" src="https://i0.wp.com/picjumbo.com/wp-content/uploads/free-stock-photos-san-francisco.jpg?w=5&amp;quality=20&amp;strip=all" title="Download The Golden Gate Bridge Sunset FREE Stock Photo" width="1080"/>]

In [30]:
'''
import requests
from bs4 import BeautifulSoup as BS

URL = 'https://pixabay.com/ko/images/search/?cat=animals'

html_pixabay_image = requests.get(URL).text
soup_pixabay_image = BS(html_pixabay_image, 'lxml')
pixabay_image_elements = soup_pixabay_image.select('img')
pixabay_image_elements

html_pixabay_image = requests.get(URL).text
soup_pixabay_image = BS(html_pixabay_image, 'lxml')
img_tags = soup_pixabay_image.find_all('a')
print(img_tags)
'''

#출력에서 보다시피 cloudflare로 인해 막힘
#https://github.com/Anorov/cloudflare-scrape - cloudflare 우회 방법인 듯함
#아래에서 직접 사용해봄

"\nimport requests\nfrom bs4 import BeautifulSoup as BS\n\nURL = 'https://pixabay.com/ko/images/search/?cat=animals'\n\nhtml_pixabay_image = requests.get(URL).text\nsoup_pixabay_image = BS(html_pixabay_image, 'lxml')\npixabay_image_elements = soup_pixabay_image.select('img')\npixabay_image_elements\n\nhtml_pixabay_image = requests.get(URL).text\nsoup_pixabay_image = BS(html_pixabay_image, 'lxml')\nimg_tags = soup_pixabay_image.find_all('a')\nprint(img_tags)\n"

In [31]:
'''
import cfscrape

scraper = cfscrape.create_scraper()
html_pixabay_image = scraper.get(URL).content
soup_pixabay_image = BS(html_pixabay_image, 'lxml')
pixabay_image_elements = soup_pixabay_image.select('a')
pixabay_image_elements[:3]
'''
#안 됨 ㅋㅋ

#https://studyforus.com/innisfree/647489
#https://stackoverflow.com/questions/18408307/how-to-extract-and-download-all-images-from-a-website-using-beautifulsoup
#검색해본 다른 해결책 / 자야해서 아직 안 해봄

"\nimport cfscrape\n\nscraper = cfscrape.create_scraper()\nhtml_pixabay_image = scraper.get(URL).content\nsoup_pixabay_image = BS(html_pixabay_image, 'lxml')\npixabay_image_elements = soup_pixabay_image.select('a')\npixabay_image_elements[:3]\n"

In [98]:
#위위위의 구글 이미지 검색으로 책 내용 계속 진행

google_image_url = google_image_elements[3].get('src')
google_image_url

'https://picjumbo.com/wp-content/uploads/free-stock-photos-san-francisco-1080x720.jpg'

In [81]:
html_image = requests.get(google_image_url)

#folder = 'C:/users/Yubin/mypy/download'
#/ 쓰면 문제 발생하나봄
folder = 'C:\\users\\Yubin\\mypy\\download'

#os.path.basename(URL)은 웹사이트나 폴더가 포함된 파일명에서 파일명만 분리하는 방법
#imageFile = open(os.path.join(folder, os.path.basename(google_image_url)), 'wb')
#google_image_url을 파일명으로 하긴 부적합한가봄

imageFile = open(os.path.join(folder, os.path.basename(google_image_url[-10:] + '.jpg')), 'wb')
#imageFile = open(os.path.join(folder, os.path.basename('giraffe_1.jpg')), 'wb')

chunk_size = 1000000
for chunk in html_image.iter_content(chunk_size):
    imageFile.write(chunk)
imageFile.close()

In [105]:
#코드 합치기 + 여러 이미지 받기
#반복 작업은 함수화
def get_image_url(url):
    html_image_url = requests.get(url).text
    soup_image_url = BS(html_image_url, 'lxml')
    image_elements = soup_image_url.select('img')
    if(image_elements != None):
        image_urls = []
        for image_element in image_elements:
            image_urls.append(image_element.get('src'))
        return image_urls
    else:
        return None

def download_image(img_folder, img_url):
    if(img_url != None):
        html_image = requests.get(img_url)
        image_File = open(os.path.join(img_folder, os.path.basename(img_url)), 'wb')
        chunk_size = 1000000
        for chunk in html_image.iter_content(chunk_size):
            imageFile.write(chunk)
        imageFile.close()
        print('이미지 파일명: {0}. 내려받기 완료!'.format(os.path.basename(img_url)))
    else:
        print('내려받을 이미지가 없습니다.')

google_url = 'https://picjumbo.com/'
figure_folder = 'C:\\users\\Yubin\\mypy\\download'
google_image_urls = get_image_url(google_url) #이미지 파일의 주소 가져오기
num_of_download_image = len(google_image_urls)

for k in range(3, num_of_download_image):
    download_image(figure_folder, google_image_urls[k])
print('=================================')
print('선택한 모든 이미지 내려받기 완료!')

ValueError: write to closed file

In [ ]:
#ValueError: write to closed file 뜨고 제대로 결과 저장 안됨
#urllib, urlretrieve 찾아보기

In [ ]:
#Web API